### KRX 데이터정보시스템
투자자별 매매동향

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import os, sys

In [2]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [3]:
driver = wd.Chrome(driver_name)
driver.set_window_position(0, 0)
driver.set_window_size(1400, 5000)

In [4]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
driver.get(main_url)
time.sleep(2)

In [5]:
# driver.find_element_by_id('tboxisuCd_finder_stkisu0_0').clear()
# time.sleep(2)
# driver.find_element_by_id('tboxisuCd_finder_stkisu0_0').send_keys('005380/현대차')
# time.sleep(2)

In [6]:
driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_0').click()
time.sleep(2)

In [7]:
driver.find_element_by_id('searchText__finder_stkisu0_0').clear()
time.sleep(2)
driver.find_element_by_id('searchText__finder_stkisu0_0').send_keys('005380/현대차')
time.sleep(2)

In [8]:
driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_0').click()

In [9]:
driver.find_element_by_css_selector('#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)').click()
time.sleep(2)

In [10]:
# Message: element not interactable Error 방지용. 클릭하기 위하여는 그 위치가 클릭할 수 있게 노출되어 있어야 함

# id가 jsOpenView_1 인 element 를 찾음
stop_tag = driver.find_element_by_id('jsOpenView_1')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [11]:
# xps = '/html/body/div[2]/section[2]/section/section/div/div/div[1]/div[1]/div[5]/button[1]'
# driver.find_element_by_xpath(xps).click() 
# # 창이 스크롤 다운할 필요가 없어야 하게 충분히 커야 함. 
# # 그렇지 않으면 Message: element not interactable Error 발생

In [12]:
driver.find_element_by_id('jsOpenView_1').click()
# interactive error 방지 error solution
time.sleep(2)

In [13]:
# driver.find_element_by_id('jsOpenView_1').send_keys(Keys.ENTER)
# # interactive error 방지 error solution

In [14]:
date = '20220630'
driver.find_element_by_id('strtDd').clear()
driver.find_element_by_id('strtDd').send_keys(date)
time.sleep(1)
driver.find_element_by_id('endDd').clear()
driver.find_element_by_id('endDd').send_keys(date)
time.sleep(1)

In [15]:
xp = '/html/body/div[2]/section[2]/section/section/div/div[2]/form/div[1]/div/a' # use full xpath
driver.find_element_by_xpath(xp).click()
time.sleep(2)

In [16]:
# res = requests.get(url, headers=headers)
df = pd.read_html(driver.page_source, attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]

In [17]:
df

,투자자구분,매도,매수,순매수,매도.1,매수.1,순매수.1
0,금융투자,76818,71856,-4962,13874093500,12964037500,-910056000
1,보험,4585,18584,13999,836020500,3380199000,2544178500
2,투신,4096,65086,60990,743027500,11829374500,11086347000
3,사모,21528,35991,14463,3892422500,6533184000,2640761500
4,은행,0,0,0,0,0,0
5,기타금융,1000,0,-1000,181000000,0,-181000000
6,연기금 등,408358,463638,55280,74137529500,84180752000,10043222500
7,기관합계,516385,655155,138770,93664093500,118887547000,25223453500
8,기타법인,4273,2614,-1659,774048500,472469500,-301579000
9,개인,788178,402966,-385212,142510577000,72765139500,-69745437500


In [18]:
index_name = ['investor', 'sell_quantity', 'buy_quantity', 'pure_buy_quantiry', 'sell', 'buy', 'pure_buy']
df.columns = index_name
df_new = df[['investor', 'sell', 'buy', 'pure_buy']]

In [19]:
df_new.set_index('investor', inplace=True)

In [20]:
dft = df_new.T

In [21]:
dft

investor,금융투자,보험,투신,사모,은행,기타금융,연기금 등,기관합계,기타법인,개인,외국인,기타외국인,전체
sell,13874093500,836020500,743027500,3892422500,0,181000000,74137529500,93664093500,774048500,142510577000,61718209000,123272000,298790200000
buy,12964037500,3380199000,11829374500,6533184000,0,0,84180752000,118887547000,472469500,72765139500,106559917500,105126500,298790200000
pure_buy,-910056000,2544178500,11086347000,2640761500,0,-181000000,10043222500,25223453500,-301579000,-69745437500,44841708500,-18145500,0


In [24]:
driver.close()
driver.quit()

#### 아래 참고

driver.find_element(By.CSS_SELECTOR, "#payment > div  > div > iframe")
and 
driver.switch_to.frame(iframe)

In [23]:
# Message: element not interactable Error 도움말
# https://stackoverflow.com/questions/56194094/how-to-fix-this-issue-element-not-interactable-selenium-python